# Tutorial 1: Azure Digital Twins

In [1]:
!pip install azure-common==1.1.28
!pip install azure-core==1.26.2
!pip install azure-digitaltwins-core==1.2.0
!pip install azure-identity==1.12.0
!pip install azure-mgmt-authorization==3.0.0
!pip install azure-mgmt-core==1.3.2
!pip install azure-mgmt-digitaltwins==6.4.0
!pip install azure-mgmt-resource==22.0.0

  Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
  Using cached azure_core-1.26.2-py3-none-any.whl (173 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl (100 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Using cached azure_digitaltwins_core-1.2.0-py3-none-any.whl (69 kB)
  Using cached msrest-0.7.1-py3-none-any.whl (85 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (

## Create Azure Digital Twins Model
- Open Azure Portal
- Navigate to the Azure Digital Twins resource
- Click on the Create Model button

<!-- ![title](images/Azure-digital-twins-setup.PNG) -->
It should show up in the model explorer as follows:

<img src="images/Azure-digital-twins-setup.PNG" width="800" height="500" />

from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.digitaltwins.core import DigitalTwinsClient

#### Azure DT Service Client

- Lets create service client to access the SDK functions for digital twin manipulation.

In [6]:
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential

In [7]:
# Replace the instance url of your digital twin here
adtInstanceUrl = "https://testdigitaltwin001.api.sea.digitaltwins.azure.net"

dt_client = DigitalTwinsClient(adtInstanceUrl, DefaultAzureCredential())
print("Service Client Created! Ready to go")

Service Client Created! Ready to go


#### Uploading a Model

- We need a model to base our digital off of. Below, we have a sample model as a `dict`. However, we can also load one from a json file.

- Microsoft provides an detailed write up about the concepts of models for digital twins.

- https://learn.microsoft.com/en-us/azure/digital-twins/concepts-models

In [8]:
sample_model = {
    "@id": "dtmi:example:SampleModel;1",
    "@type": "Interface",
    "displayName": "SampleModel",
    "contents": [
        {
            "@type": "Relationship",
            "name": "contains"
        },
        {
            "@type": "Property",
            "name": "data",
            "schema": "string"
        },
        {
            "@type": "Telemetry",
            "name": "sensor",
            "schema": "double"      
        }
    ],
    "@context": "dtmi:dtdl:context;2"
}

- It's good practice to put our call for model creation inside a try-except statement. This is because we cannot upload two models with the same id and subsequent calls with the models having the same id will throw an error.

In [9]:
try:
    # put all the models that we want to create into a list
    models = [sample_model]
    dt_client.create_models(models)
    print("Model uploaded to the instance")
except Exception as e:
    print(e)

Model uploaded to the instance


<!-- ![title](images/model_upload.png) -->
It should show up in the model explorer as follows:

<img src="images/model_upload.png" width="800" height="500" />

#### Creating a digital twin
- Now that we have uploaded a model, we can use it to create a digital twin out of it.

In [14]:
# unique id of the digital twin
digital_twin_id = 'digiTwin-1'

# the necessary metadata of the digital twin
temp_twin = {
    "$metadata": {
        "$model": "dtmi:example:SampleModel;1" #give the model's id here
    },
    "$dtId": digital_twin_id,
    "data": "Hello World"
}
created_twin = dt_client.upsert_digital_twin(digital_twin_id, temp_twin)
print("DT Created")

DT Created


#### Publish Telemetry

In [15]:
# update property of dt
digital_twin_id = "digiTwin-1"
telemetry_payload = '{"sensor": 10.1}'

# Create the telemetry message
telemetry_message = {
    "data": telemetry_payload
}

dt_client.publish_telemetry(digital_twin_id,telemetry_payload)

Datetime with no tzinfo will be considered UTC.
